In [ ]:
# Google Colab setup
from google.colab import drive
drive.mount('/content/drive')

# Optional: Unzip your dataset if needed
import zipfile
import os
from pathlib import Path

ZIP_PATH = "/content/drive/MyDrive/ChessData.zip"  # Update if needed
EXTRACT_TO = "/content/ChessData"

if os.path.exists(ZIP_PATH):
    print(f"Extracting {ZIP_PATH}...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_TO)
    print(f"Extracted to {EXTRACT_TO}")
else:
    print(f"Warning: {ZIP_PATH} not found. Please upload ChessData.zip to Google Drive.")

# Install required packages
!pip install torch torchvision tqdm matplotlib

# Imports
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from datetime import datetime
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# CONFIG
DATA_DIR = "/content/ChessData/data2"  # Update if needed
AUG_MODE = 1
BATCH_SIZE = 32
LR = 0.0005
EPOCHS = 25
IMG_SIZE = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# AUGMENTATION PIPELINES
def get_transforms(mode):
    if mode == 0:
        train_tf = transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
        ])
    elif mode == 1:
        train_tf = transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ])
    elif mode == 2:
        train_tf = transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.RandomRotation(25),
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2),
            transforms.RandomPerspective(distortion_scale=0.4, p=0.4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ])
    test_tf = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
    ])
    return train_tf, test_tf

train_tf, test_tf = get_transforms(AUG_MODE)

# DATASET LOADING
train_data = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_tf)
test_data  = datasets.ImageFolder(os.path.join(DATA_DIR, "test"),  transform=test_tf)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_data,  batch_size=BATCH_SIZE, shuffle=False)
CLASS_NAMES = train_data.classes
print("Classes:", CLASS_NAMES)

# CNN MODEL
class ChessCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 12 * 12, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 3)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


model = ChessCNN().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# TRAINING + VALIDATION LOOP
best_acc = 0.0
train_losses = []
val_accuracies = []
for epoch in range(1, EPOCHS + 1):
    model.train()
    train_loss = 0
    print(f"\nEpoch {epoch}/{EPOCHS}")
    progress_bar = tqdm(train_loader, desc="Training", leave=False)
    for imgs, labels in progress_bar:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    train_losses.append(train_loss/len(train_loader))
    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = correct / total
    val_accuracies.append(val_acc)
    print(f"Epoch {epoch}/{EPOCHS} | Train Loss: {train_loss/len(train_loader):.4f} | Test Acc: {val_acc*100:.2f}%")
    # Save best model to Google Drive
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "/content/drive/MyDrive/best_model_colab.pth")
        print(f"✔ Best model updated! (Acc: {val_acc*100:.2f}%)")
print("Training complete. Best accuracy =", best_acc*100, "%")

# Plot training history
plt.figure(figsize=(10,4))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_accuracies, label='Val Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.title('Training Loss & Validation Accuracy')
plt.grid(True)
plt.savefig('/content/drive/MyDrive/training_history_colab.png')
plt.show()
